In [218]:
import pandas as pd
import sqlalchemy as db
import cx_Oracle as orcl
import datetime as dt
import numpy as np

print("cx_Oracle.version:", orcl.version)
# show all columns without without truncating
pd.set_option('display.max_columns', None)
import sys
reload(sys)
sys.setdefaultencoding('utf8')


('cx_Oracle.version:', '7.2.3')


In [ ]:
host = "10.21.18.15"
port = "1521"
sid = "PROFITSDB"
dsn = orcl.makedsn(host, port, sid)
connection = orcl.connect("mobileuser","Kenya1234##", dsn)
cur = orcl.Cursor(connection) 


In [2]:
host = "10.21.18.13"
port = "1521"
sid = "ECHODB1"
dsn = orcl.makedsn(host, port, sid)
connection = orcl.connect("ALLAN.ASWANI","Password1!", dsn)
cur = orcl.Cursor(connection) 


In [219]:
host="128.2.5.46"
port="1521"
sid="PROFITSDB"
username="MOBILEUSER"
password="nathan"
dsn_tns = orcl.makedsn(host, port,sid)
connection = orcl.connect(user=username, password=password, dsn=dsn_tns)
cur = orcl.Cursor(connection) 

##checking customer information
statements = """SELECT * FROM CUSTOMER"""
customer = pd.read_sql_query(statements, connection)
pd.set_option('display.max_columns', None)
customer.head(4)

###latest customer numbers
customer.tail(4)

statement_trx = """select P.ACCOUNT_NUMBER,p.cust_id,
trim(C.SURNAME) || ' '|| trim(C.FIRST_NAME) ||' ' || trim(C.MIDDLE_NAME) as name,
D.ID_TRANSACT,
d.TIMESTAMP,
D.ID_JUSTIFIC,
J.DESCRIPTION,
DECODE (D.DEBIT_CREDIT_FLAG,'2','CR','1','DR') "DR/CR",
D.ENTRY_AMOUNT,
D.PREV_ACC_BALANCE, 
D.VALUE_DATE,
D.TRX_USR,
D.ENTRY_COMMENTS
from fst_demand_extrait d, justific j, profits_account p, customer c
where
D.ID_JUSTIFIC = J.ID_JUSTIFIC
and
P.CUST_ID = C.CUST_ID
and
D.FK_DEPOSIT_ACCOACC= P.DEP_ACC_NUMBER
and p.PRFT_SYSTEM=3
and trunc(d.TIMESTAMP) >= to_date('01-06-2019','dd-mm-yyyy')
and trunc(d.TIMESTAMP) <= to_date('30-09-2019','dd-mm-yyyy')
order by D.VALUE_DATE asc"""
trxns_data = pd.read_sql_query(statement_trx, connection)
pd.set_option('display.max_columns', None)
trxns_data.head(10)


trxns_data.tail(10)

In [221]:
statement_trx_4 = '''select D.TMSTAMP,
D.TRX_DATE,
d.TRX_UNIT,
d.TRX_USR,
d.TRX_USR_SN,
d.TUN_INTERNAL_SN,
trim(to_char(D.TRX_DATE,'yyyymmdd'))||''||trim(D.TRX_UNIT)||''||trim(D.TRX_USR)||''||trim (abs(D.TRX_USR_SN) -1)||''||trim(D.TUN_INTERNAL_SN) as TRANSACTION_REF,
P.ID_PRODUCT,
P.DESCRIPTION AS PRODUCT_DESCRIPTION,
R.ACCOUNT_NUMBER, 
A.FK_CUSTOMERCUST_ID,
C.ID_NO AS NATIONAL_ID,
S.MOBILE_TEL,
A.DESIGNATION AS ACCOUNT_NAME,
D.I_ID_JUSTIFIC,
J.DESCRIPTION AS TRANSACTION_TYPE,
ABS (D.I_AMOUNT) "TRANSACTION_AMOUNT",
D.ACC_AMOUNT_40 AS COMMISSION,
D.ACC_AMOUNT_42 AS "EXCISE DUTY",
D.O_FINAL_ACC_AMOUNT AS TOTAL_TRX_AMOUNT,
D.I_COMMENTS TRX_COMMENTS,
F.REVERSE_FLAG, 
F.REVERSED_TRX_FLAG

from prod.DEP_TRX_RECORDING D, prod.JUSTIFIC J, prod.PRODUCT P, prod.DEPOSIT_ACCOUNT A, prod.PROFITS_ACCOUNT R, prod.FST_DEMAND_EXTRAIT F, prod.OTHER_ID C, prod.CUSTOMER S 
WHERE 
A.ACCOUNT_NUMBER = D.I_ACCOUNT_NUMBER
AND R.DEP_ACC_NUMBER = D.I_ACCOUNT_NUMBER
AND R.DEP_ACC_NUMBER = F.FK_DEPOSIT_ACCOACC
and A.ACCOUNT_NUMBER = F.FK_DEPOSIT_ACCOACC
AND C.FK_CUSTOMERCUST_ID=A.FK_CUSTOMERCUST_ID
AND D.ID_PRODUCT = P.ID_PRODUCT
AND D.I_ID_JUSTIFIC = J.ID_JUSTIFIC
AND A.FK_CUSTOMERCUST_ID=S.CUST_ID
AND F.TRX_DATE = D.TRX_DATE   
AND F.TRX_UNIT  = D.TRX_UNIT 
AND F.TRX_SN = D.TRX_USR_SN 
AND F.TRX_USR = D.TRX_USR
and F.TUN_INTERNAL_SN = D.TUN_INTERNAL_SN 
AND TRUNC(D.TMSTAMP) between TO_DATE('01-01-2019', 'dd-mm-yyyy') and 
TO_DATE('25-12-2019', 'dd-mm-yyyy') 
--AND D.I_ID_JUSTIFIC IN (31804,31805,31815,32096,32097,32099,32096,32097,32099) 
AND R.PRFT_SYSTEM = 3
AND R.PRODUCT_ID NOT IN ('36000','35000','34000')
AND F.REVERSE_FLAG =0
AND F.REVERSED_TRX_FLAG=0
'''
trxns_data_trx_4 = pd.read_sql_query(statement_trx_4, connection)
trxns_data_trx_4

,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,ID_PRODUCT,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,EXCISE DUTY,TOTAL_TRX_AMOUNT,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG
0,2019-01-01 04:00:42,2018-12-31,16,99973118,63,1,201812311699973118621,31110,EMPLOYER PROVIDENT ACCOUNT,4186413101,418641,21783803,254722564736,JUDY WANGUI MWANGI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,137.33,,0,0
1,2019-01-01 04:00:42,2018-12-31,16,99973118,63,1,201812311699973118621,31110,EMPLOYER PROVIDENT ACCOUNT,4186413101,418641,21783803,254722564736,JUDY WANGUI MWANGI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,137.33,,0,0
2,2019-01-01 00:40:58,2018-12-31,100,99974431,7095,1,201812311009997443170941,31572,VIRTUAL ACCOUNT MOBILE,4449343201,444934,27161560,254716801532,ESTHER NG'ANG'A ...,34006,STANDING ORDER PAYMENT,5.25,0.0,0.0,-5.25,LOANS STANDING ORDER.74431-0600043404,0,0
3,2019-01-01 00:41:32,2018-12-31,100,99974431,7395,1,201812311009997443173941,31572,VIRTUAL ACCOUNT MOBILE,4449413201,444941,32131713,254715554386,SIPHIRA MUHIA ...,34006,STANDING ORDER PAYMENT,15.31,0.0,0.0,-15.31,LOANS STANDING ORDER.74431-0600043572,0,0
4,2019-01-01 00:39:10,2018-12-31,100,99974431,6183,1,201812311009997443161821,31572,VIRTUAL ACCOUNT MOBILE,4469103201,446910,13326017,254723516353,GEORGE ONGERA ...,34006,STANDING ORDER PAYMENT,34.00,0.0,0.0,-34.00,LOANS STANDING ORDER.74431-0600042890,0,0
5,2019-01-01 00:35:30,2018-12-31,100,99974431,5010,1,201812311009997443150091,31572,VIRTUAL ACCOUNT MOBILE,4470923201,447092,21170950,254710231206,MILLYCENT OTIENO ...,34006,STANDING ORDER PAYMENT,81.17,0.0,0.0,-81.17,LOANS STANDING ORDER.74431-0600042221,0,0
6,2019-01-01 04:01:56,2018-12-31,25,99973116,686,1,2018123125999731166851,31715,CROSS OVER SAVING ACCOUNT,2223193101,222319,11673852,254720970988,ANNE WAWIRA MUCHANGI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,62.87,,0,0
7,2019-01-01 04:01:56,2018-12-31,25,99973116,686,1,2018123125999731166851,31715,CROSS OVER SAVING ACCOUNT,2223193101,222319,11673852,254720970988,ANNE WAWIRA MUCHANGI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,62.87,,0,0
8,2019-01-01 04:03:58,2018-12-31,22,99973116,1563,1,20181231229997311615621,31105,TREASURE SAVINGS ACCOUNT,2362383101,236238,0191584404,254729602802,JAYDEN KINGORI C/O MERCY MWARANIA WAMBUI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,11.22,,0,0
9,2019-01-01 04:01:56,2018-12-31,510,99973116,685,1,20181231510999731166841,31715,CROSS OVER SAVING ACCOUNT,2360763101,236076,28231827,254729924135,"EVA,WILSON,FRANCIS AND MARY ...",93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,211.80,,0,0


trxns_data_trx_4 = pd.read_csv("/Users/allan.aswani/Downloads/calculation/transactions_2019_11_25.csv")

In [326]:
trxns_data_trx_4

,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,ID_PRODUCT,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,EXCISE DUTY,TOTAL_TRX_AMOUNT,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG
0,2019-01-01 04:00:42,2018-12-31,16,99973118,63,1,201812311699973118621,31110,EMPLOYER PROVIDENT ACCOUNT,4186413101,418641,21783803,254722564736,JUDY WANGUI MWANGI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,137.33,,0,0
1,2019-01-01 04:00:42,2018-12-31,16,99973118,63,1,201812311699973118621,31110,EMPLOYER PROVIDENT ACCOUNT,4186413101,418641,21783803,254722564736,JUDY WANGUI MWANGI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,137.33,,0,0
2,2019-01-01 00:40:58,2018-12-31,100,99974431,7095,1,201812311009997443170941,31572,VIRTUAL ACCOUNT MOBILE,4449343201,444934,27161560,254716801532,ESTHER NG'ANG'A ...,34006,STANDING ORDER PAYMENT,5.25,0.0,0.0,-5.25,LOANS STANDING ORDER.74431-0600043404,0,0
3,2019-01-01 00:41:32,2018-12-31,100,99974431,7395,1,201812311009997443173941,31572,VIRTUAL ACCOUNT MOBILE,4449413201,444941,32131713,254715554386,SIPHIRA MUHIA ...,34006,STANDING ORDER PAYMENT,15.31,0.0,0.0,-15.31,LOANS STANDING ORDER.74431-0600043572,0,0
4,2019-01-01 00:39:10,2018-12-31,100,99974431,6183,1,201812311009997443161821,31572,VIRTUAL ACCOUNT MOBILE,4469103201,446910,13326017,254723516353,GEORGE ONGERA ...,34006,STANDING ORDER PAYMENT,34.00,0.0,0.0,-34.00,LOANS STANDING ORDER.74431-0600042890,0,0
5,2019-01-01 00:35:30,2018-12-31,100,99974431,5010,1,201812311009997443150091,31572,VIRTUAL ACCOUNT MOBILE,4470923201,447092,21170950,254710231206,MILLYCENT OTIENO ...,34006,STANDING ORDER PAYMENT,81.17,0.0,0.0,-81.17,LOANS STANDING ORDER.74431-0600042221,0,0
6,2019-01-01 04:01:56,2018-12-31,25,99973116,686,1,2018123125999731166851,31715,CROSS OVER SAVING ACCOUNT,2223193101,222319,11673852,254720970988,ANNE WAWIRA MUCHANGI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,62.87,,0,0
7,2019-01-01 04:01:56,2018-12-31,25,99973116,686,1,2018123125999731166851,31715,CROSS OVER SAVING ACCOUNT,2223193101,222319,11673852,254720970988,ANNE WAWIRA MUCHANGI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,62.87,,0,0
8,2019-01-01 04:03:58,2018-12-31,22,99973116,1563,1,20181231229997311615621,31105,TREASURE SAVINGS ACCOUNT,2362383101,236238,0191584404,254729602802,JAYDEN KINGORI C/O MERCY MWARANIA WAMBUI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,11.22,,0,0
9,2019-01-01 04:01:56,2018-12-31,510,99973116,685,1,20181231510999731166841,31715,CROSS OVER SAVING ACCOUNT,2360763101,236076,28231827,254729924135,"EVA,WILSON,FRANCIS AND MARY ...",93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,211.80,,0,0


In [282]:
df1 = trxns_data_trx_4
df1

,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,ID_PRODUCT,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,EXCISE DUTY,TOTAL_TRX_AMOUNT,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG
0,2019-01-01 04:00:42,2018-12-31,16,99973118,63,1,201812311699973118621,31110,EMPLOYER PROVIDENT ACCOUNT,4186413101,418641,21783803,254722564736,JUDY WANGUI MWANGI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,137.33,,0,0
1,2019-01-01 04:00:42,2018-12-31,16,99973118,63,1,201812311699973118621,31110,EMPLOYER PROVIDENT ACCOUNT,4186413101,418641,21783803,254722564736,JUDY WANGUI MWANGI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,137.33,,0,0
2,2019-01-01 00:40:58,2018-12-31,100,99974431,7095,1,201812311009997443170941,31572,VIRTUAL ACCOUNT MOBILE,4449343201,444934,27161560,254716801532,ESTHER NG'ANG'A ...,34006,STANDING ORDER PAYMENT,5.25,0.0,0.0,-5.25,LOANS STANDING ORDER.74431-0600043404,0,0
3,2019-01-01 00:41:32,2018-12-31,100,99974431,7395,1,201812311009997443173941,31572,VIRTUAL ACCOUNT MOBILE,4449413201,444941,32131713,254715554386,SIPHIRA MUHIA ...,34006,STANDING ORDER PAYMENT,15.31,0.0,0.0,-15.31,LOANS STANDING ORDER.74431-0600043572,0,0
4,2019-01-01 00:39:10,2018-12-31,100,99974431,6183,1,201812311009997443161821,31572,VIRTUAL ACCOUNT MOBILE,4469103201,446910,13326017,254723516353,GEORGE ONGERA ...,34006,STANDING ORDER PAYMENT,34.00,0.0,0.0,-34.00,LOANS STANDING ORDER.74431-0600042890,0,0
5,2019-01-01 00:35:30,2018-12-31,100,99974431,5010,1,201812311009997443150091,31572,VIRTUAL ACCOUNT MOBILE,4470923201,447092,21170950,254710231206,MILLYCENT OTIENO ...,34006,STANDING ORDER PAYMENT,81.17,0.0,0.0,-81.17,LOANS STANDING ORDER.74431-0600042221,0,0
6,2019-01-01 04:01:56,2018-12-31,25,99973116,686,1,2018123125999731166851,31715,CROSS OVER SAVING ACCOUNT,2223193101,222319,11673852,254720970988,ANNE WAWIRA MUCHANGI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,62.87,,0,0
7,2019-01-01 04:01:56,2018-12-31,25,99973116,686,1,2018123125999731166851,31715,CROSS OVER SAVING ACCOUNT,2223193101,222319,11673852,254720970988,ANNE WAWIRA MUCHANGI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,62.87,,0,0
8,2019-01-01 04:03:58,2018-12-31,22,99973116,1563,1,20181231229997311615621,31105,TREASURE SAVINGS ACCOUNT,2362383101,236238,0191584404,254729602802,JAYDEN KINGORI C/O MERCY MWARANIA WAMBUI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,11.22,,0,0
9,2019-01-01 04:01:56,2018-12-31,510,99973116,685,1,20181231510999731166841,31715,CROSS OVER SAVING ACCOUNT,2360763101,236076,28231827,254729924135,"EVA,WILSON,FRANCIS AND MARY ...",93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.00,0.0,0.0,211.80,,0,0


In [283]:
df1.ACCOUNT_NUMBER = df1.ACCOUNT_NUMBER.astype('int64')

In [284]:
account_100 = df1.query('ACCOUNT_NUMBER == ""')
account_100

/Users/allan.aswani/etl/lib/python2.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,ID_PRODUCT,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,EXCISE DUTY,TOTAL_TRX_AMOUNT,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG


In [214]:
#column names and type
df1.dtypes.to_list

<bound method Index.tolist of Index([u'TMSTAMP', u'TRX_DATE', u'TRX_UNIT', u'TRX_USR', u'TRX_USR_SN',
       u'TUN_INTERNAL_SN', u'TRANSACTION_REF', u'PRODUCT_DESCRIPTION',
       u'ACCOUNT_NUMBER', u'ID_PRODUCT', u'FK_CUSTOMERCUST_ID', u'NATIONAL_ID',
       u'MOBILE_TEL', u'ACCOUNT_NAME', u'I_ID_JUSTIFIC', u'TRANSACTION_TYPE',
       u'TRANSACTION_AMOUNT', u'COMMISSION', u'TRX_COMMENTS', u'REVERSE_FLAG',
       u'REVERSED_TRX_FLAG'],
      dtype='object')>

In [287]:
#staff accounts 
df2 = df1.query('ID_PRODUCT == "31111"' or 'ID_PRODUCT == "31705"' or 'ID_PRODUCT == "31200"') 

df2.sort_values(ascending=False,by='TMSTAMP')
df2

,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,ID_PRODUCT,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,EXCISE DUTY,TOTAL_TRX_AMOUNT,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG
1007,2019-01-01 04:01:10,2018-12-31,200,99973117,371,1,20181231200999731173701,31111,STAFF SALARY ACCOUNT,3438813101,343881,31243328,254701561835,ANNE ACHIENG ONYANGO ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.0,0.0,0.0,1.38,,0,0
1008,2019-01-01 04:01:10,2018-12-31,200,99973117,371,1,20181231200999731173701,31111,STAFF SALARY ACCOUNT,3438813101,343881,31243328,254701561835,ANNE ACHIENG ONYANGO ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.0,0.0,0.0,1.38,,0,0
1713,2019-01-01 04:01:10,2018-12-31,200,99973117,367,1,20181231200999731173661,31111,STAFF SALARY ACCOUNT,3429773201,342977,31485537,254707137333,TRACY MORO ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.0,0.0,0.0,20.83,,0,0
1714,2019-01-01 04:01:10,2018-12-31,200,99973117,367,1,20181231200999731173661,31111,STAFF SALARY ACCOUNT,3429773201,342977,31485537,254707137333,TRACY MORO ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.0,0.0,0.0,20.83,,0,0
1740,2019-01-01 04:01:10,2018-12-31,200,99973117,370,1,20181231200999731173691,31111,STAFF SALARY ACCOUNT,3267643101,326764,27417482,254797051069,MARTIN GUAMA KIBOI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.0,0.0,0.0,67.58,,0,0
1741,2019-01-01 04:01:10,2018-12-31,200,99973117,370,1,20181231200999731173691,31111,STAFF SALARY ACCOUNT,3267643101,326764,27417482,254797051069,MARTIN GUAMA KIBOI ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.0,0.0,0.0,67.58,,0,0
2625,2019-01-01 04:23:07,2018-12-31,200,99973111,29717,1,2018123120099973111297161,31111,STAFF SALARY ACCOUNT,2000064815,97640,14676542,254721935992,PRISCA MUTIO MUTINDA ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.0,0.0,0.0,59.06,,0,0
2626,2019-01-01 04:23:07,2018-12-31,200,99973111,29717,1,2018123120099973111297161,31111,STAFF SALARY ACCOUNT,2000064815,97640,14676542,254721935992,PRISCA MUTIO MUTINDA ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.0,0.0,0.0,59.06,,0,0
2636,2019-01-01 04:22:16,2018-12-31,200,99973111,29237,1,2018123120099973111292361,31111,STAFF SALARY ACCOUNT,2000032959,25926,3642260.,254721750439,LUGANJE FLORENCE MASHAKA ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.0,0.0,0.0,615.54,,0,0
2637,2019-01-01 04:22:16,2018-12-31,200,99973111,29237,1,2018123120099973111292361,31111,STAFF SALARY ACCOUNT,2000032959,25926,3642260.,254721750439,LUGANJE FLORENCE MASHAKA ...,93989,FIRST DEMAND INTEREST CALCULATION UPDATE,0.0,0.0,0.0,615.54,,0,0


In [288]:
#df1['TRX_USR'].unique().tolist()

In [290]:
#staff accounts 
#df_test = df1.query('PRODUCT_DESCRIPTION == "STAFF SALARY ACCOUNT"') 
#df2.TMSTAMP.sort_values(ascending = False)

In [291]:
df3 = df2.query('TRX_USR.str.contains("KOCE")' or 'TRX_USR.str.contains("ESB")')
df3

,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,ID_PRODUCT,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,EXCISE DUTY,TOTAL_TRX_AMOUNT,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG
91838,2019-01-01 00:02:17,2019-01-02,109,KOCE0001,135,1,20190102109KOCE00011341,31111,STAFF SALARY ACCOUNT,2000106497,171127,24437517,,JULIUS MUNGAI MUIGAI ...,32097,PAY BILL(ACCOUNT TO MPESA PAYBILL),1500.00,0.0,0.0,-1500.00,MPESA B2B PAY BILL 841063 VERONA,0,0
91839,2019-01-01 08:14:39,2019-01-02,109,KOCE0001,1938,1,20190102109KOCE000119371,31111,STAFF SALARY ACCOUNT,2000106497,171127,24437517,,JULIUS MUNGAI MUIGAI ...,32100,UTILITY BILL PAYMENT(APP),3500.00,0.0,0.0,-3500.00,BILL PAYMENT TO KPLC_PRE 14245959060,0,0
91842,2019-01-01 00:31:27,2019-01-02,109,KOCE0001,304,1,20190102109KOCE00013031,31111,STAFF SALARY ACCOUNT,2000107304,172768,28487961,254726059806,FELIX OGO BALESA ...,32099,ACCOUNT TO MPESA(B2C),700.00,0.0,0.0,-700.00,SEND TO MPESA 254726059806,0,0
91849,2019-01-01 06:45:11,2019-01-02,109,KOCE0001,1153,1,20190102109KOCE000111521,31111,STAFF SALARY ACCOUNT,2000109702,178026,28486112,254714538554,GICHIA WAITHIRA SAMSON ...,32098,AIRTIME PURCHASE( MOBILE APP),50.00,0.0,0.0,-50.00,BUY AIRTIME FOR AIRTSAF 254714538554,0,0
91970,2019-01-01 00:47:57,2019-01-02,109,KOCE0001,381,1,20190102109KOCE00013801,31111,STAFF SALARY ACCOUNT,4425303201,442530,24062378,254720256476,BELINDA KEITANY ...,31815,CR FROM MOBILE BANKING-MPESA TO ACC,30000.00,0.0,0.0,30000.00,NA13VORRX1-254720256476 MPESA TO ACC,0,0
92391,2019-01-01 00:05:25,2019-01-02,109,KOCE0001,155,1,20190102109KOCE00011541,31111,STAFF SALARY ACCOUNT,2000038766,50023,4440692,254723681065,JANE MARGARET ADHIAMBO SURUNGAI ...,32099,ACCOUNT TO MPESA(B2C),100.00,0.0,0.0,-100.00,SEND TO MPESA 254723681065,0,0
92420,2019-01-01 07:36:01,2019-01-02,109,KOCE0001,1484,1,20190102109KOCE000114831,31111,STAFF SALARY ACCOUNT,2000061953,94031,22916777,254722586861,BILLIET ANGELA KIKECH ...,32099,ACCOUNT TO MPESA(B2C),3000.00,0.0,0.0,-3000.00,SEND TO MPESA 254722586861,0,0
92427,2019-01-01 07:54:44,2019-01-02,109,KOCE0001,1712,1,20190102109KOCE000117111,31111,STAFF SALARY ACCOUNT,2000067004,100091,23675274,254720711124,JANE WAITHIRA GITHENYA ...,32100,UTILITY BILL PAYMENT(APP),600.00,0.0,0.0,-600.00,BILL PAYMENT TO KPLC_PRE 22170439305,0,0
92434,2019-01-01 07:03:12,2019-01-02,109,KOCE0001,1216,1,20190102109KOCE000112151,31111,STAFF SALARY ACCOUNT,2000079280,115119,22720334,0722222024,ARTHUR GACHONDE MAINA ...,32099,ACCOUNT TO MPESA(B2C),100.00,0.0,0.0,-100.00,SEND TO MPESA 254722222024,0,0
92446,2019-01-01 08:25:32,2019-01-02,109,KOCE0001,2085,1,20190102109KOCE000120841,31111,STAFF SALARY ACCOUNT,2000081600,118066,12777450,254727175120,NANCY WANGUI GITHAIGA ...,32099,ACCOUNT TO MPESA(B2C),2000.00,0.0,0.0,-2000.00,SEND TO MPESA 254727175120,0,0


In [292]:
#period of running 
#df3.TRX_DATE#.max()

#df2.TRX_DATE =  pd.to_datetime(df2.TRX_DATE, infer_datetime_format=True)
#df3.TMSTAMP.sort_values(ascending=False)
#df3

In [293]:
df3.dtypes

TMSTAMP                datetime64[ns]
TRX_DATE               datetime64[ns]
TRX_UNIT                        int64
TRX_USR                        object
TRX_USR_SN                      int64
TUN_INTERNAL_SN                 int64
TRANSACTION_REF                object
ID_PRODUCT                      int64
PRODUCT_DESCRIPTION            object
ACCOUNT_NUMBER                  int64
FK_CUSTOMERCUST_ID              int64
NATIONAL_ID                    object
MOBILE_TEL                     object
ACCOUNT_NAME                   object
I_ID_JUSTIFIC                   int64
TRANSACTION_TYPE               object
TRANSACTION_AMOUNT            float64
COMMISSION                    float64
EXCISE DUTY                   float64
TOTAL_TRX_AMOUNT              float64
TRX_COMMENTS                   object
REVERSE_FLAG                   object
REVERSED_TRX_FLAG              object
dtype: object

In [295]:
NOW = df3.TRX_DATE.max()
rfmTable = df3.groupby('ACCOUNT_NUMBER').agg({'TRX_DATE': lambda x: (NOW - x.max()).days, 'TRANSACTION_REF': lambda x: len(x), 'COMMISSION': lambda x: x.sum()})
rfmTable['TRX_DATE'] = rfmTable['TRX_DATE'].astype(int)
rfmTable.rename(columns={'TRX_DATE': 'recency', 
                         'TRANSACTION_REF': 'frequency', 
                         'COMMISSION': 'monetary_value'}, inplace=True)

In [296]:
rfmTable

,frequency,recency,monetary_value
ACCOUNT_NUMBER,,,
201643101,11,0,0.0
949563101,703,0,4731.0
1098683102,703,2,5640.0
1302963102,155,154,555.0
1445193102,1,94,0.0
1591173102,3,108,66.0
1602073103,1,69,0.0
1619473101,1,188,0.0
1637003102,43,0,486.0


In [297]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
print(quantiles)
quantiles = quantiles.to_dict()
quantiles

      frequency  recency  monetary_value
0.25       20.0      0.0             0.0
0.50       83.0      7.0           528.0
0.75      252.0     30.0          1815.0


{'frequency': {0.25: 20.0, 0.5: 83.0, 0.75: 252.0},
 'monetary_value': {0.25: 0.0, 0.5: 528.0, 0.75: 1815.0},
 'recency': {0.25: 0.0, 0.5: 7.0, 0.75: 30.0}}

In [298]:
#reference https://towardsdatascience.com/find-your-best-customers-with-customer-segmentation-in-python-61d602f9eee6
segmented_rfm = rfmTable

In [299]:
#The lowest recency, highest frequency and monetary amounts are our best customers.
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [300]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()


,frequency,recency,monetary_value,r_quartile,f_quartile,m_quartile
ACCOUNT_NUMBER,,,,,,
201643101,11,0,0.0,1,4,4
949563101,703,0,4731.0,1,1,1
1098683102,703,2,5640.0,2,1,1
1302963102,155,154,555.0,4,2,2
1445193102,1,94,0.0,4,4,4


In [301]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str)   + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,frequency,recency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
ACCOUNT_NUMBER,,,,,,,
201643101,11,0,0.0,1,4,4,144
949563101,703,0,4731.0,1,1,1,111
1098683102,703,2,5640.0,2,1,1,211
1302963102,155,154,555.0,4,2,2,422
1445193102,1,94,0.0,4,4,4,444


In [302]:
segmented_rfm.RFMScore.max()

'444'

In [303]:
staff_identify = pd.read_excel (r'/Users/allan.aswani/Documents/scorecardworks/staff_loans_version_1.xlsx', sheet_name='Sheet2')
staff_identify

,Pfno,Name,ACCOUNT,MOBILE,Loan Limit,Type of whizz Account,cust_id,target saving,fixed saving,opening date,grade,income proxy,accountNumber,cbsCustomerID,customerID,customerNames,idNumber,mobileNumber,productCode
0,363,Lucy Wangui Ndungu,2000032981,722310293,6000,STAFF SALARY ACCOUNT,29624.0,0.0,0.0,2000-12-06,01,10000.0,2.000033e+09,29624.0,519577.0,LUCY WANGUI,8515233,2.547223e+11,31111.0
1,390,Rospa Jepngetich Kandie,2000033147,722466794,6000,STAFF SALARY ACCOUNT,30258.0,0.0,0.0,2018-06-13,UNC,NaN,2.000033e+09,30258.0,544942.0,ROSPA JEPNG'ETICH,9778506,2.547225e+11,31111.0
2,403,Alice Wanjiru Weru,2000035405,254722338270,6000,STAFF SALARY ACCOUNT,9582.0,0.0,0.0,2001-10-04,02,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,415,George Mburu Karanu,2000032902,722559933,18000,STAFF SALARY ACCOUNT,4496.0,0.0,0.0,2000-11-29,M3,30000.0,2.000033e+09,4496.0,92.0,GEORGE MBURU,10874594,2.547226e+11,31111.0
4,436,Eric Kiptoo Kosgei,2000033074,722859902,6000,STAFF SALARY ACCOUNT,1440.0,0.0,0.0,2012-11-17,UNC,NaN,2.000033e+09,1440.0,128.0,KOSGEI ERIC,9328141,2.547229e+11,31111.0
5,439,David Kimutai Kirui,2000033024,721249078,6000,STAFF SALARY ACCOUNT,21135.0,0.0,0.0,2000-12-06,02,10000.0,2.000033e+09,21135.0,1278.0,DAVID KIRUI,10314300,2.547212e+11,31111.0
6,441,Esther Wambui Muya,2000033121,720269385,6000,STAFF SALARY ACCOUNT,21123.0,0.0,0.0,2016-08-12,UNC,NaN,2.000033e+09,21123.0,2276.0,ESTHER MUYA,10269408,2.547203e+11,31111.0
7,450,Jane Wambui Kuria,2000032960,722633539,12000,STAFF SALARY ACCOUNT,21439.0,0.0,0.0,2000-12-05,M2,20000.0,2.000033e+09,21439.0,181.0,JANE KURIA,10880731,2.547226e+11,31111.0
8,462,Gordon Mbigo Nyagudi,2000033031,714606788,18000,STAFF SALARY ACCOUNT,29488.0,0.0,0.0,2014-10-28,M3,30000.0,2.000033e+09,29488.0,29.0,GORDON NYAGUDI,8182751,2.547146e+11,31111.0
9,463,Lucy Njoki Njogu,2000033205,722790605,12000,STAFF SALARY ACCOUNT,20997.0,0.0,0.0,2000-12-13,M2,20000.0,2.000033e+09,20997.0,136.0,LUCY NJOGU,10087038,2.547228e+11,31111.0


In [304]:
segmented_rfm = pd.DataFrame(segmented_rfm)
segmented_rfm.reset_index(level=0, inplace=True)
segmented_rfm

,ACCOUNT_NUMBER,frequency,recency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
0,201643101,11,0,0.0,1,4,4,144
1,949563101,703,0,4731.0,1,1,1,111
2,1098683102,703,2,5640.0,2,1,1,211
3,1302963102,155,154,555.0,4,2,2,422
4,1445193102,1,94,0.0,4,4,4,444
5,1591173102,3,108,66.0,4,4,3,443
6,1602073103,1,69,0.0,4,4,4,444
7,1619473101,1,188,0.0,4,4,4,444
8,1637003102,43,0,486.0,1,3,3,133
9,1687573101,1,70,0.0,4,4,4,444


In [305]:
staff_identify.dtypes

Pfno                             object
Name                             object
ACCOUNT                           int64
MOBILE                           object
Loan Limit                        int64
Type of whizz Account            object
cust_id                         float64
target saving                   float64
fixed saving                    float64
opening date             datetime64[ns]
grade                            object
income proxy                    float64
accountNumber                   float64
cbsCustomerID                   float64
customerID                      float64
customerNames                    object
idNumber                         object
mobileNumber                    float64
productCode                     float64
dtype: object

In [306]:
print(segmented_rfm)

     ACCOUNT_NUMBER  frequency  recency  monetary_value  r_quartile  \
0         201643101         11        0             0.0           1   
1         949563101        703        0          4731.0           1   
2        1098683102        703        2          5640.0           2   
3        1302963102        155      154           555.0           4   
4        1445193102          1       94             0.0           4   
5        1591173102          3      108            66.0           4   
6        1602073103          1       69             0.0           4   
7        1619473101          1      188             0.0           4   
8        1637003102         43        0           486.0           1   
9        1687573101          1       70             0.0           4   
10       1718983101          3      266             0.0           4   
11       1850443102         12      284             0.0           4   
12       1933933101        164        2          1656.0           2   
13    

In [307]:
staff_account_limit = staff_identify.merge(segmented_rfm,left_on='ACCOUNT', right_on="ACCOUNT_NUMBER", how = "left")

In [308]:
staff_identify

,Pfno,Name,ACCOUNT,MOBILE,Loan Limit,Type of whizz Account,cust_id,target saving,fixed saving,opening date,grade,income proxy,accountNumber,cbsCustomerID,customerID,customerNames,idNumber,mobileNumber,productCode
0,363,Lucy Wangui Ndungu,2000032981,722310293,6000,STAFF SALARY ACCOUNT,29624.0,0.0,0.0,2000-12-06,01,10000.0,2.000033e+09,29624.0,519577.0,LUCY WANGUI,8515233,2.547223e+11,31111.0
1,390,Rospa Jepngetich Kandie,2000033147,722466794,6000,STAFF SALARY ACCOUNT,30258.0,0.0,0.0,2018-06-13,UNC,NaN,2.000033e+09,30258.0,544942.0,ROSPA JEPNG'ETICH,9778506,2.547225e+11,31111.0
2,403,Alice Wanjiru Weru,2000035405,254722338270,6000,STAFF SALARY ACCOUNT,9582.0,0.0,0.0,2001-10-04,02,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,415,George Mburu Karanu,2000032902,722559933,18000,STAFF SALARY ACCOUNT,4496.0,0.0,0.0,2000-11-29,M3,30000.0,2.000033e+09,4496.0,92.0,GEORGE MBURU,10874594,2.547226e+11,31111.0
4,436,Eric Kiptoo Kosgei,2000033074,722859902,6000,STAFF SALARY ACCOUNT,1440.0,0.0,0.0,2012-11-17,UNC,NaN,2.000033e+09,1440.0,128.0,KOSGEI ERIC,9328141,2.547229e+11,31111.0
5,439,David Kimutai Kirui,2000033024,721249078,6000,STAFF SALARY ACCOUNT,21135.0,0.0,0.0,2000-12-06,02,10000.0,2.000033e+09,21135.0,1278.0,DAVID KIRUI,10314300,2.547212e+11,31111.0
6,441,Esther Wambui Muya,2000033121,720269385,6000,STAFF SALARY ACCOUNT,21123.0,0.0,0.0,2016-08-12,UNC,NaN,2.000033e+09,21123.0,2276.0,ESTHER MUYA,10269408,2.547203e+11,31111.0
7,450,Jane Wambui Kuria,2000032960,722633539,12000,STAFF SALARY ACCOUNT,21439.0,0.0,0.0,2000-12-05,M2,20000.0,2.000033e+09,21439.0,181.0,JANE KURIA,10880731,2.547226e+11,31111.0
8,462,Gordon Mbigo Nyagudi,2000033031,714606788,18000,STAFF SALARY ACCOUNT,29488.0,0.0,0.0,2014-10-28,M3,30000.0,2.000033e+09,29488.0,29.0,GORDON NYAGUDI,8182751,2.547146e+11,31111.0
9,463,Lucy Njoki Njogu,2000033205,722790605,12000,STAFF SALARY ACCOUNT,20997.0,0.0,0.0,2000-12-13,M2,20000.0,2.000033e+09,20997.0,136.0,LUCY NJOGU,10087038,2.547228e+11,31111.0


In [309]:
staff_account_limit

,Pfno,Name,ACCOUNT,MOBILE,Loan Limit,Type of whizz Account,cust_id,target saving,fixed saving,opening date,grade,income proxy,accountNumber,cbsCustomerID,customerID,customerNames,idNumber,mobileNumber,productCode,ACCOUNT_NUMBER,frequency,recency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
0,363,Lucy Wangui Ndungu,2000032981,722310293,6000,STAFF SALARY ACCOUNT,29624.0,0.0,0.0,2000-12-06,01,10000.0,2.000033e+09,29624.0,519577.0,LUCY WANGUI,8515233,2.547223e+11,31111.0,2.000033e+09,142.0,9.0,1716.0,3.0,2.0,2.0,322
1,390,Rospa Jepngetich Kandie,2000033147,722466794,6000,STAFF SALARY ACCOUNT,30258.0,0.0,0.0,2018-06-13,UNC,NaN,2.000033e+09,30258.0,544942.0,ROSPA JEPNG'ETICH,9778506,2.547225e+11,31111.0,2.000033e+09,18.0,14.0,0.0,3.0,4.0,4.0,344
2,403,Alice Wanjiru Weru,2000035405,254722338270,6000,STAFF SALARY ACCOUNT,9582.0,0.0,0.0,2001-10-04,02,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000035e+09,1.0,233.0,0.0,4.0,4.0,4.0,444
3,415,George Mburu Karanu,2000032902,722559933,18000,STAFF SALARY ACCOUNT,4496.0,0.0,0.0,2000-11-29,M3,30000.0,2.000033e+09,4496.0,92.0,GEORGE MBURU,10874594,2.547226e+11,31111.0,2.000033e+09,231.0,0.0,1611.0,1.0,2.0,2.0,122
4,436,Eric Kiptoo Kosgei,2000033074,722859902,6000,STAFF SALARY ACCOUNT,1440.0,0.0,0.0,2012-11-17,UNC,NaN,2.000033e+09,1440.0,128.0,KOSGEI ERIC,9328141,2.547229e+11,31111.0,2.000033e+09,455.0,21.0,2841.0,3.0,1.0,1.0,311
5,439,David Kimutai Kirui,2000033024,721249078,6000,STAFF SALARY ACCOUNT,21135.0,0.0,0.0,2000-12-06,02,10000.0,2.000033e+09,21135.0,1278.0,DAVID KIRUI,10314300,2.547212e+11,31111.0,2.000033e+09,132.0,3.0,1083.0,2.0,2.0,2.0,222
6,441,Esther Wambui Muya,2000033121,720269385,6000,STAFF SALARY ACCOUNT,21123.0,0.0,0.0,2016-08-12,UNC,NaN,2.000033e+09,21123.0,2276.0,ESTHER MUYA,10269408,2.547203e+11,31111.0,2.000033e+09,770.0,0.0,6642.0,1.0,1.0,1.0,111
7,450,Jane Wambui Kuria,2000032960,722633539,12000,STAFF SALARY ACCOUNT,21439.0,0.0,0.0,2000-12-05,M2,20000.0,2.000033e+09,21439.0,181.0,JANE KURIA,10880731,2.547226e+11,31111.0,2.000033e+09,476.0,0.0,2232.0,1.0,1.0,1.0,111
8,462,Gordon Mbigo Nyagudi,2000033031,714606788,18000,STAFF SALARY ACCOUNT,29488.0,0.0,0.0,2014-10-28,M3,30000.0,2.000033e+09,29488.0,29.0,GORDON NYAGUDI,8182751,2.547146e+11,31111.0,2.000033e+09,945.0,0.0,3840.0,1.0,1.0,1.0,111
9,463,Lucy Njoki Njogu,2000033205,722790605,12000,STAFF SALARY ACCOUNT,20997.0,0.0,0.0,2000-12-13,M2,20000.0,2.000033e+09,20997.0,136.0,LUCY NJOGU,10087038,2.547228e+11,31111.0,2.000033e+09,212.0,16.0,1590.0,3.0,2.0,2.0,322


In [310]:
staff_account_limit.query('RFMScore>0')

,Pfno,Name,ACCOUNT,MOBILE,Loan Limit,Type of whizz Account,cust_id,target saving,fixed saving,opening date,grade,income proxy,accountNumber,cbsCustomerID,customerID,customerNames,idNumber,mobileNumber,productCode,ACCOUNT_NUMBER,frequency,recency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
0,363,Lucy Wangui Ndungu,2000032981,722310293,6000,STAFF SALARY ACCOUNT,29624.0,0.0,0.0,2000-12-06,01,10000.0,2.000033e+09,29624.0,519577.0,LUCY WANGUI,8515233,2.547223e+11,31111.0,2.000033e+09,142.0,9.0,1716.0,3.0,2.0,2.0,322
1,390,Rospa Jepngetich Kandie,2000033147,722466794,6000,STAFF SALARY ACCOUNT,30258.0,0.0,0.0,2018-06-13,UNC,NaN,2.000033e+09,30258.0,544942.0,ROSPA JEPNG'ETICH,9778506,2.547225e+11,31111.0,2.000033e+09,18.0,14.0,0.0,3.0,4.0,4.0,344
2,403,Alice Wanjiru Weru,2000035405,254722338270,6000,STAFF SALARY ACCOUNT,9582.0,0.0,0.0,2001-10-04,02,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000035e+09,1.0,233.0,0.0,4.0,4.0,4.0,444
3,415,George Mburu Karanu,2000032902,722559933,18000,STAFF SALARY ACCOUNT,4496.0,0.0,0.0,2000-11-29,M3,30000.0,2.000033e+09,4496.0,92.0,GEORGE MBURU,10874594,2.547226e+11,31111.0,2.000033e+09,231.0,0.0,1611.0,1.0,2.0,2.0,122
4,436,Eric Kiptoo Kosgei,2000033074,722859902,6000,STAFF SALARY ACCOUNT,1440.0,0.0,0.0,2012-11-17,UNC,NaN,2.000033e+09,1440.0,128.0,KOSGEI ERIC,9328141,2.547229e+11,31111.0,2.000033e+09,455.0,21.0,2841.0,3.0,1.0,1.0,311
5,439,David Kimutai Kirui,2000033024,721249078,6000,STAFF SALARY ACCOUNT,21135.0,0.0,0.0,2000-12-06,02,10000.0,2.000033e+09,21135.0,1278.0,DAVID KIRUI,10314300,2.547212e+11,31111.0,2.000033e+09,132.0,3.0,1083.0,2.0,2.0,2.0,222
6,441,Esther Wambui Muya,2000033121,720269385,6000,STAFF SALARY ACCOUNT,21123.0,0.0,0.0,2016-08-12,UNC,NaN,2.000033e+09,21123.0,2276.0,ESTHER MUYA,10269408,2.547203e+11,31111.0,2.000033e+09,770.0,0.0,6642.0,1.0,1.0,1.0,111
7,450,Jane Wambui Kuria,2000032960,722633539,12000,STAFF SALARY ACCOUNT,21439.0,0.0,0.0,2000-12-05,M2,20000.0,2.000033e+09,21439.0,181.0,JANE KURIA,10880731,2.547226e+11,31111.0,2.000033e+09,476.0,0.0,2232.0,1.0,1.0,1.0,111
8,462,Gordon Mbigo Nyagudi,2000033031,714606788,18000,STAFF SALARY ACCOUNT,29488.0,0.0,0.0,2014-10-28,M3,30000.0,2.000033e+09,29488.0,29.0,GORDON NYAGUDI,8182751,2.547146e+11,31111.0,2.000033e+09,945.0,0.0,3840.0,1.0,1.0,1.0,111
9,463,Lucy Njoki Njogu,2000033205,722790605,12000,STAFF SALARY ACCOUNT,20997.0,0.0,0.0,2000-12-13,M2,20000.0,2.000033e+09,20997.0,136.0,LUCY NJOGU,10087038,2.547228e+11,31111.0,2.000033e+09,212.0,16.0,1590.0,3.0,2.0,2.0,322


In [311]:
def limitAdjuster(x):
    if x in ('111','112','122','133','134','144'):
        return 1.25
    elif x in ('211','221','222','232','233','243','244'):
        return 1.5
    elif x in ('311','321','322','333','343','344'): 
        return 1.75
    elif x in ('422','433','444'): 
        return 2
    else:
        return 1

In [312]:
#,'111','112','122','133','134','144','211','221','222','232','233','243','244','311','321','322','333','343','344','422','433','444'

In [313]:
staff_account_limit['Loan Limit'] = staff_account_limit['Loan Limit'].astype('int64')

In [314]:

#staff_account_limit.RFMScore.unique #staff_account_limit.RFMScore.fillna(0, inplace = True) 

In [315]:
#staff_account_limit.RFMScore.unique

In [316]:
staff_account_limit['adjuster'] = staff_account_limit.RFMScore.apply(limitAdjuster)

In [317]:
staff_account_limit["new_limit_24_Nov"] = staff_account_limit['Loan Limit'] *staff_account_limit['adjuster']

In [254]:
staff_account_limit.head()
staff_account_limit_nov_2019 = staff_account_limit[["Pfno", "Name","ACCOUNT","Loan Limit" ,"new_limit_24_Nov"]]

In [318]:
staff_account_limit[staff_account_limit.ACCOUNT_NUMBER.isnull()]

,Pfno,Name,ACCOUNT,MOBILE,Loan Limit,Type of whizz Account,cust_id,target saving,fixed saving,opening date,grade,income proxy,accountNumber,cbsCustomerID,customerID,customerNames,idNumber,mobileNumber,productCode,ACCOUNT_NUMBER,frequency,recency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore,adjuster,new_limit_24_Nov
47,1154,Carolyne Mutegi Munene,2000077019,254722394554,6000,STAFF SALARY ACCOUNT,112296.0,0.0,0.0,2009-08-26,02,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6000.0
48,1156,Kenneth Nzomo Kiilu,7050000013,254723639144,18000,CURRENT ACCOUNT STAFF,113495.0,0.0,0.0,2010-03-08,M3,30000.0,7.050000e+09,113495.0,1207.0,KENNETH KIILU,13429771,2.547236e+11,31705.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,18000.0
53,1174,Mark Gichuru Kinyuru,7050000070,254722932606,18000,CURRENT ACCOUNT STAFF,114704.0,0.0,0.0,2012-05-10,M3,30000.0,7.050000e+09,114704.0,122.0,MARK KINYURU,11608297,2.547229e+11,31705.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,18000.0
74,1244,Amos Ndati Ndinza,2000089446,718786064,6000,STAFF SALARY ACCOUNT,128194.0,0.0,0.0,2011-09-07,02,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6000.0
81,1273,Stephen Paul Kimaku Wainaina,7050000162,722826853,18000,CURRENT ACCOUNT STAFF,136298.0,0.0,0.0,2012-11-14,M3,30000.0,7.050000e+09,136298.0,1185.0,STEPHEN PAUL,23149980,2.547228e+11,31705.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,18000.0
88,1326,Annrita Mwirigi,7050000186,NaN,6000,NaN,NaN,NaN,NaN,NaT,O1,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6000.0
89,1327,Samuel Muthomi Kibiti,7050000187,254728068518,6000,CURRENT ACCOUNT STAFF,138242.0,0.0,0.0,2013-01-15,03,10000.0,7.050000e+09,138242.0,43.0,SAMUEL MUTHOMI KIBITI,23707644,2.547281e+11,31705.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6000.0
91,1329,Eric Kihu Nguithi,7050000195,254728216722,6000,CURRENT ACCOUNT STAFF,138820.0,0.0,0.0,2013-02-11,03,10000.0,7.050000e+09,138820.0,36.0,ERIC NGUITHI,25641336,2.547282e+11,31705.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6000.0
93,1334,Lawi Omondi Ochaye,7050000194,254728039840,6000,CURRENT ACCOUNT STAFF,138819.0,0.0,0.0,2013-02-11,02,10000.0,7.050000e+09,138819.0,1906.0,LAWI OCHAYE,25030674,2.547280e+11,31705.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6000.0
94,1355,Francis Mungathia Miriti,7050000214,254725333982,6000,CURRENT ACCOUNT STAFF,140753.0,0.0,0.0,2013-04-15,03,10000.0,7.050000e+09,140753.0,120.0,FRANCIS MUNGATHIA,23180519,2.547205e+11,31705.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6000.0


In [319]:
staff_account_limit.ACCOUNT_NUMBER.isnull().sum()

157

In [325]:
staff_account_limit.query('idNumber == 24991210')

,Pfno,Name,ACCOUNT,MOBILE,Loan Limit,Type of whizz Account,cust_id,target saving,fixed saving,opening date,grade,income proxy,accountNumber,cbsCustomerID,customerID,customerNames,idNumber,mobileNumber,productCode,ACCOUNT_NUMBER,frequency,recency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore,adjuster,new_limit_24_Nov
206,1917,Anthony Mwenda Mutembei,1934193201,254720935672,12000,CURRENT ACCOUNT STAFF,193419.0,0.0,0.0,2016-02-16,M2,20000.0,1.934193e+09,193419.0,1146.0,ANTHONY MUTEMBEI,24991210,2.547228e+11,31705.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12000.0


import psycopg2
t
    connection = psycopg2.connect(user = "datawarehouse",
                                  password = "P@ssw0rd",
                                  host = "128.2.4.15",
                                  port = "5432",
                                  database = "datawarehouse")

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
            
            
            dbname = "datawarehouse",
                 host = "128.2.4.15", port = 5432,
                 user = "datawarehouse", password = pw)

In [ ]:
import psycopg2
    connection = psycopg2.connect(user = "datawarehouse",
                                  password = "P@ssw0rd",
                                  host = "128.2.4.15",
                                  port = "5432",
                                  database = "datawarehouse")

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")


In [323]:
export_csv = staff_account_limit.to_csv (r'/Users/allan.aswani/Documents/scorecardworks/staff_account_limit_25_Nov_2019.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


In [322]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')